# Assignment 3: Tiền xử lý văn bản với tiếng Việt
Tổng quan: ở bài tập này chúng ta xây dựng chương trình tiền xử lý văn bản với tiếng Việt.

Import các thư viện cần dùng. Lưu ý ở đây ta, ta dùng thư viện underthesea để dùng tokenize tiếng Việt, để cài đặt các bạn làm theo hướng dẫn sau ([link](https://github.com/undertheseanlp/underthesea))

In [2]:
import os,glob
import codecs
import sys
import re
from underthesea import word_tokenize

## Câu hỏi 1: tạo corpus và khảo sát dữ liệu
Dữ liệu trong phần này được trích một phần từ bộ dữ liệu [VNTC](https://github.com/duyvuleo/VNTC). VNTC là bộ dữ liệu về tin tức tiếng Việt với nhiều chủ đề khác nhau. Trong nội dung phần này ta chỉ xử lý cho chủ đề khoa học trong VNTC. Ta sẽ tạo một corpus từ cả thư mục train và test. Hoàn thiện đoạn chương trình:
- ghi sentences_list ra 1 file với tên dataset_name.txt, mỗi phần tử là một doc, sẽ trên 1 dòng.
- Kiểm tra xem trong corpus có bao nhiêu docs

In [3]:
dataset_name = "VNTC_khoahoc"
path = ['./VNTC_khoahoc/Train_Full/', './VNTC_khoahoc/Test_Full/']

if os.listdir(path[0]) == os.listdir(path[1]): 
    folder_list = [os.listdir(path[0]), os.listdir(path[1])]
    print("train labels = test labels")
else:
    print("train labels differ from test labels")

doc_num = 0
sentences_list = []
meta_data_list = []
for i in range(2): 
    for folder_name in folder_list[i]:
        folder_path = path[i] + folder_name
        if folder_name[0] != ".":
            for file_name in glob.glob(os.path.join(folder_path, '*.txt')):
                f = codecs.open(file_name, 'br')
                file_content = (f.read().decode("utf-16")).replace("\r\n", " ")
                sentences_list.append(file_content.strip()) 
                f.close
                doc_num += 1

print("Number of documents: ", doc_num)

train labels = test labels
Number of documents:  3916


## Câu hỏi 2: Viết các hàm tiền xử lý
### Câu hỏi 2.1: Viết hàm làm sạch văn bản.
Gợi ý:
- Biết rằng văn bản chỉ giữ lại các ký tự sau: aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0-9(),!?\'\`
- Sau đó cắt các khoảng trắng của văn bản đầu vào.

In [4]:
import re

def clean_str(text):
    """
    Cleans the input text by keeping only the specified characters and trimming whitespace.
    
    Parameters:
    text (str): The input text to be cleaned.
    
    Returns:
    str: The cleaned text.
    """
    # Define the regex pattern to keep only the specified characters
    pattern = r"[^aAàÀảẢãÃáÁạẠăĂằẰẳẲẵẴắẮặẶâÂầẦẩẨẫẪấẤậẬbBcCdDđĐeEèÈẻẺẽẼéÉẹẸêÊềỀểỂễỄếẾệỆfFgGhHiIìÌỉỈĩĨíÍịỊjJkKlLmMnNoOòÒỏỎõÕóÓọỌôÔồỒổỔỗỖốỐộỘơƠờỜởỞỡỠớỚợỢpPqQrRsStTuUùÙủỦũŨúÚụỤưƯừỪửỬữỮứỨựỰvVwWxXyYỳỲỷỶỹỸýÝỵỴzZ0-9(),!?\'\`]"
    
    # Replace unwanted characters with a space
    cleaned_text = re.sub(pattern, ' ', text)
    
    # Trim whitespace
    cleaned_text = cleaned_text.strip()
    
    return cleaned_text

# Example usage
sample_text = "Hello World! This is a test. 1234567890 àÀảẢãÃáÁạẠăĂ"
cleaned_sample = clean_str(sample_text)
print(cleaned_sample)

Hello World! This is a test  1234567890 àÀảẢãÃáÁạẠăĂ


## Câu hỏi 2.2: Viết hàm chuyển văn bản về dạng thường

In [5]:
# make all text lowercase
def text_lowercase(string):
    return string.lower()

## Câu hỏi 2.3: tách từ
Gợi ý: sử dụng hàm word_tokenize() đã import ở trên với hai tham số lần lượt là strings, và format="text".

In [6]:
def tokenize(strings):
    return word_tokenize(strings, format="text")

## Câu hỏi 2.4 Loại bỏ từ dừng
Ở đây để loại bỏ từ dừng, ta dùng một danh sách các từ dừng tiếng Việt. Chúng được lưu trong file './vietnamese-stopwords.txt'. Hoàn thiện chương trình sau:
- Đối chiếu từng từ trong văn bản (strings) nếu từ nào không có trong từ dừng thì thêm vào doc_words

In [10]:
def remove_stopwords(strings):
    strings = strings.split()
    f = open('./vietnamese-stopwords.txt', 'r', encoding='utf-8')  
    stopwords = f.readlines()
    stop_words = [s.replace("\n", '') for s in stopwords]
    doc_words = []
    for word in strings:
        if word not in stop_words:
            doc_words.append(word)
    doc_str = ' '.join(doc_words).strip()

    return doc_str

## Câu hỏi 2.5 xây dựng hàm tiền xử lý
Gợi ý: gọi lần lượt các hàm clean_str, text_lowercase, tokenize, remove_stopwords, rồi trả ra kết quả cho hàm.

In [11]:
def text_preprocessing(strings):
    strings = clean_str(strings)
    strings = text_lowercase(strings)
    strings = tokenize(strings)
    strings = remove_stopwords(strings)
    return strings

## Câu hỏi 3: thực hiện tiền xử lý
Công việc giờ chúng ta sẽ đọc corpus từ file đã làm ở câu hỏi 1. Sau đó ta sẽ gọi hàm tiền xử lý cho từng doc trong corpus trên.

Gợi ý: gọi hàm text_preprocessing() với tham số đầu vào là doc_content, lưu vào biến temp1

In [12]:
doc_content_list = []

for doc in sentences_list:
    doc_content_list.append(text_preprocessing(doc))

clean_docs = []
for doc_content in doc_content_list:
    temp1 = doc_content
    clean_docs.append(temp1)
print("\nlength of clean_docs = ", len(clean_docs))
print('clean_docs[0]:\n' + clean_docs[0])


length of clean_docs =  3916
clean_docs[0]:
ninh thuận_địa_điểm ưu_tiên nhà_máy điện hạt_nhân góc ninh_thuận , địa_điểm ưu_tiên lựa_chọn nhà_máy điện hạt_nhân vương_hữu , viện trưởng viện năng_lượng nguyên_tử việt_nam , địa_điểm hiện cân_nhắc lựa_chọn nhà_máy điện hạt_nhân ninh_thuận , phú_yên phan_rang , ninh_thuận địa_điểm ưu_tiên lựa_chọn tiến_triển dự_án xây_dựng nhà_máy điện hạt_nhân vn , vương_hữu , viện năng_lượng nguyên_tử việt_nam hoàn_tất dự_án tiền_khả_thi trình chính_phủ phê_duyệt dự_kiến , ( 2007 ) , dự_án khả_thi đáp_ứng kịp tiến_độ xây_dựng nhà_máy điện hạt_nhân vn kế_hoạch , nhà_máy điện hạt_nhân xây_dựng đi hoạt_động 2020 quy_mô công_suất 2 000 mw 4 000 mw , chiếm 5 9 tổng_công_suất phát_điện toàn_quốc lựa_chọn công_nghệ nhà_máy điện hạt_nhân vn , viện năng_lượng nguyên_tử việt_nam , hiện phân_tích trưng_cầu ý_kiến chuyên_gia , công_nghệ nhà_máy điện hạt_nhân chọn_lựa cơ_sở an_toàn vận_hành hướng_dẫn cơ_quan năng_lượng nguyên_tử quốc_tế ( iaea ) , nhà_máy điện hạt_nhâ

## Câu 4: ghi dữ liệu đã tiền xử lý
Gợi ý: ghi clean_docs vào trong file có tên dataset_name + '.clean.txt', trong đó mỗi doc (phần tử trong clean_docs) ghi trên một dòng.

In [14]:
clean_corpus_str = '\n'.join(clean_docs)
f = open(dataset_name + '.clean.txt', 'w', encoding='utf-8')
f.write(clean_corpus_str)
f.close()